In [1]:
# Add Spring Security in Resume

### What was Role and Responsibility

### 1. DM977 CorFormulary 

1. Got FRD from onsite
2. Create Query Tracker, HLD and sent onsite for review/approval multiple times

- Populate the correct format of data file in excel sheet and database.
- Add two extra column names ESP and ESP Manufaccturer Name in existing format for Download template, Upload File, Download full data.

###### Existing Design :
- Download Template : NDC, Labeler Code, Action. (there is no ESP and ESP_Manufaccturer_Name columns)
- Upload File : NDC, Labeler Code, Action. (there is no ESP and ESP_Manufaccturer_Name columns)
- Download full Data : NDC, Labeler Code, Drug Description, Manufacturer Name. (there is no ESP and ESP_Manufaccturer_Name columns)

###### New Design :
- Download Template : NDC, Labeler Code, ESP, ESP Manufacturer Name, Action. (ESP: dropdown with value YES and NO. ESP Manufaccturer Name : dropdown and its value will be fetched from CFG_CODE_DECODE table)
- Upload File : NDC, Labeler Code, ESP, ESP Manufacturer Name, Action.
- Download full Data : NDC, Labeler Code, Drug Description, Manufacturer Name, ESP, ESP Manufacturer Name.

3. Written SQL query for Download Template, Upload File, Download Full Data.
4. perfrom DBCR and DMR(pythonscript run on jupyter for large data migration)
5. Check on SonarLint for code quality, Imlemented JUnit, Sonarqube Analysis
6. Written trigger query
6. Create proper document for UTC and UTR.
7. DM977CorFormularyImages : folder in mobile


### 2. DM989 : Java BTSBatch

###### Existing Design :

- There are currently two files data file and sig file from stores, It .sig and datafile.txt have many information which is not usable as of now, which takes more time to run batch.

###### New Design :

- As per new design 340b will receive zip folder contain txt file from each stores/SIMS which having orders.
- 340b will unzip that zipped folder at a destination path/directory, then will process those individual text file received from stores.
- SIMSListener will parse the acknowledgement files in the directory from SIMS/store and populate the data structure and return map of PO Id as key and list of SIMSListenerBeans as value as it was doing earlier.
- Data inside the files will be pipe delimited: Order number, product identifier, location type, order qty, order creation date time.

###### Batch run steps :

- first pull request R99 -> R99_UAT, check it is deployed then run in putty.
- open putty : enter host aamitlndd1la003.walgreens.com, login as 340bqa and pswrd 
- pwd
- cd script directory  (script directly is where script file is present in WinSCP)
- sh ... .sh

###### Code work :

- first it will fetch poID which is already present in database.
- Create method for zipFile(String filePath, String zipFileName) and unZipFile(String filepath) in POUtil.java
- Some constant directory are wriiten in CommonConstants.java
- Some zip directory are mentioned in BTSBatch.properties_UAT
- added logger in catch block also, SIMSListenerServiceIMPL.java -> startService()
- parseFile() : PO id as key and list of SIMSListenerBean as value, 
- loadSIMSListenerBean(dataToParse) : for each line data in text form, it will used in parseFile(), it will return SIMSLIStenerbean.

### Services : Party Service, Ocis Service, Search Orchestrator Service, IPNI Service, Income Detail Consumer


###### Party Service :

- Party Service provides the required capabilities for creating, updating and retrieving information related to party.
- POST : Onboard a new party, onboard a new minor party along with guardian/parent 
- Party LifeCycle :
 - GET : /parties/{partyId} - Return party Details
 - GET : /parties/{partyId}/statuses - Return all party statuses
 - GET : /parties/{partyId}/addresses - Return all party addresses
 - GET : /parties/{partyId}/contactDetails - Return all party contact details
 - GET : /parties/{partyId}/portfolioItems - Return all portfolio items
 - GET : /parties/{partyId}/preferences - Fetch preferences for a party brand
 - GET : /parties/{partyId}/notificationBrandPreference - Fetch notification brand preference for a party
 - GET : /parties/{partyId}/countryAssociation - Returns all party country associations
 - GET : /parties/{partyId}/countryAssociation/{countryAssociationId} - Returns all party country associations
 - GET : /parties/{partyId}/identifiers - Get all party identifier
 - GET : /parties/{partyId}/employmentDetails - get all party employment details
 - GET : /parties/{partyId}/partyInternalReference - get all party internal reference
 
- Migration :
 - POST : /parties/migrate/basicinfo - API for preloading basic info of a party
 - POST : /parties/{partyId}migrate - API for updating party detailsfor migration

- Subscriber :
 - CreateOrUpdateAccountSubscriber - 
 - CreatePortfolioSubscriber
 - ClosePartySubscriber - The subscriber update the party lifecycle status to close
 - closeAccounntSubscriber - The subscriber sets party's portfolio item to closed
 - UpdateContactDetailSubscriber - The Subscriber update part's contact details

- BDD Testing : It contains the acceptance test scenarios for BDD feature testing of various party service and update the results in Cucumber reports in jenkin pipeline
 
###### Ocis Service : 

- This Ocis Service provides api's to integrate with any OCIS functionalities. Currently we have lending group API only. 
- POST : /customer/lending-groups - Create Lending Group
- PUT : /customers/lending-group - Update Lending Group


###### Search Orchestrator Service :

- Search Orchestrator Service provides api's to support party prospect journey (CCS, nCino Requirements Shaping - CDP)
- Prospect and Onboarding journey api's : 
 - POST : /parties/{sourceId}/non-personal - Non-Personal party Additional Details
 - GET : /parties/search/non-personal - 
 - POST : /parties/search/personal - Personal party search
 - Get : /parties/{partyId}/personal - Personal Party Additional Details
 - POST : /parties/{partyID}/source-Identifiers - Party Update Ncino Id to Ocis

###### IPNI Service :

- This is micro service which have different consumer which will listed to stage 4 downstream create and update message. Consumer will internally interact with the party-service to get the party related information, curate the message (json) and publish to GCP kafka. The IP&I will listen to GCP kafka for the required messages

 - Telephone (create and update message)
 - individual : personal party (update message)
 - Country association (create and update message)
 - Address (create and update message)
 - Party internal refrence (create message)
 - Additional party attribute (create and update message)
 
- .\gradlew bootrun, .\gradlew build

###### Income Detail Consumer : 

- No controller
- Kafka, Service, Repository


### IMDT-78632 : IPRD, Create API, Add a flag to differentiate the calls from CCS and others

###### Requirement : 

- Changes in Services : Party Service, SearchOrchestrator Service (.quryParam() changes)
- call from CCS : retrieve IPR details by joining the tables
- call from non-CCS : retreve only from IPRD (which ActiveCustomerRelationshipDetails)
- Changes : 
 - Added @RequestParam(name ="source", required = false) String source, (in CustomerRelationshipHierarchyController.java)
 - getCustomer() data with extra parameter with source "CCS"
 - added if check : for source in ServiceImpl
 - added Constant Data String SOURCE_CCS = "CCS"
 - Updated Swagger specs as well : api-specs->party-service.yml (other yml files also)
 - Written Unit Test Cases : CustomerDatawithJoin and CustomerDataWithoutJoin, also changes in testData
 - Changes in Repository Layer, Written SQL queries for both condition as per requirement.
 - Controller -> Service (Interface) -> ServiceImpl (also extra method calling) -> Repository (.java files for sql queries with @Query())

### IMDT-84085 EmploymentDetails, Issue when update with empty string for EmployerName and JobTitle

- Changes in PartyService /..../ EmploymentDetailServiceIMPL.java
- getEmploymentDetailResponse() { persistemploymentDetail() }
- In getEmploymentDetailResponse() there will be only non null values
- persistemploymentDetail() will return EmploymentDetail (@Table)
- First we create Party, then update with some string value and then update with empty string
- Expected : EmployerName & Jobtitle should not appear in the api for empty string update
- Issue was when we update in 2nd time with empty string, it should update in dataBase but after hitting the api for retriew employmentDetails it should not appear
- Changes in persistemploymentDetail() where if condition check, for both null and empty string it will set null so that in getEmploymentDetailResponse(), the null values will not appear in api, Earlier there was only check for null.
-
- for consumer, in which company consumer are working, Employer Name like details.....

### IMDT-83962 EmploymentDetails, CreatedTimestamp value getting replaced/overlapped by the LastModifiedTimestamp on update operation

- changes in Service : income detail consumer service, party service
- /customer/consumer/service/NonCorePartyDetailMessageFactory.java --> ConstructEmploymentDetailsAvroMessage(nonCorePartyDetailMessageWrapper)
- added extra if check for eventType if it is insert or update, if it is insert then only set employmentdetails.setCreatedTimestammp() and .setcreatedBy()
- Separate method for constructTimeStampField() of String return type 

### IMDT-82987 Date of Death is not updating IPNI individual topic and in DataBase with valid value and with empty string,

- src/main/java/.../ipni/util/IpniRequestFactory.java --> ConstructIndividualUpdateDetail()
- IN IPNIRequestFactory.java: The code was not written for set the value of DeathDate, Only for DeathNotifiedDate was written
- PartyLifecycleStatusCode Deceased,
- individual.setDeathDate(constructDate(personDetails.getDateOfDeath()))
- written extra unit test case also

### IMDT-83317 Update DOB as empty string for personal party is not updating in individual topic and in Data Base

- if check was missing for empty string for update
- previous check was like update DOB with valid DOB in which the empty string case was not included
- written extra unit test case also

### CDSL-718 : Disable Unused endPoints from Ocis-Service

###### Requirement :

- env : bld, int, pre, prd 
- only bld and int env are allowed to have all endpoints active at this time
- disble each api except which is related to lending group and update with party-internal-reference
- api : get and update for partyDetails, PartyTelephonesTelephones, GetAddressDetails
- Removed swagger for temporary disabling endpoints
- other ways : Disable each api separately using @ConditionalOnProperty with proper name and mention in application.proerties
- Ideal way also move all the endpoonts needed by data shield to a new controller and disable it.

# Major Project JBDL :
- create user, getuser, jpa, spring authentication, kafka streaming publish message kafka producer, consume message, save data in database, transactionService

- in utils comman constant, common library, common code, which will be part of multiple repositories, multiple application
- in pm.xml : dependencies
- factory : connection to kafka, construct,
- kafka : kafka producer, for publishing the event which will listened by other consumer
- add loggers in proper places
- exceptional handling
- global security configuration
- individual as topic
- unit testing for individual module
-
- transactionservice : when the transaction is initiated it will make the entry in transaction tablewith the status as "pending" and it will publish the event, this event will be published on transactionCreatedTopic will be consume by wallet service to perform the validation and update the wallet status and again it will publish the event which will be consumed by transaction service to update the transaction status. Once the transaction status is updated either it will publish again two events 1st will be transaction success events which will publish for sender of the transaction and 2nd event will be published only if the transaction is successfull which is for the receiver ends, so receiver will published only if the transaction is successsfull otherwise only sender events will be published.